**Install python dependencies**

In [21]:
!pip install -q -r ./requirements.txt

**Load python libraries**

In [32]:
from sqlalchemy import create_engine
from os import getenv
import requests
import pandas as pd
from statistics import median


**Connect to database**

In [39]:
db_name = getenv("POSTGRES_DB")
db_user = getenv("POSTGRES_USER")
db_pass = getenv("POSTGRES_PASSWORD")
db_host = 'localhost'
db_port = '5432'
db_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_user, db_pass, db_host, db_port, db_name)
db = create_engine(db_string)

In [35]:
ODDS_API_URL = "https://api.the-odds-api.com/v3/odds/"
ODDS_API_KEY = getenv("ODDS_API_KEY")

In [34]:
res = requests.get(ODDS_API_URL, params={
    'api_key': ODDS_API_KEY,
    'region': 'eu',
    'sport': 'soccer_germany_bundesliga'
})

odds = res.json()

NameError: name 'ODDS_API_URL' is not defined

In [33]:
column_names = ["team", "odd_win", "odd_lose", "odd_draw"]
df_odds = pd.DataFrame(columns = column_names)

matches = odds['data']

for match in matches:
    team_1 = match['teams'][0]
    team_2 = match['teams'][1]

    odds_team1, odds_team2, odds_draw = [], [], []

    betting_sites = match['sites']

    for site in betting_sites:
        # key can be h2h or h2h_lay
        for key in site['odds']:
             odds_team1.append(site['odds'][key][0])
             odds_team2.append(site['odds'][key][1])
             odds_draw.append(site['odds'][key][2])

    df_odds = df_odds.append({'team': team_1, 'odd_win': median(odds_team1), 'odd_lose': median(odds_team2), 'odd_draw': median(odds_draw)}, ignore_index=True)
    df_odds = df_odds.append({'team': team_2, 'odd_win': median(odds_team2), 'odd_lose': median(odds_team1), 'odd_draw': median(odds_draw)}, ignore_index=True)

,team,odd_win,odd_lose,odd_draw
0,FC Koln,1.360,8.000,5.800
1,Schalke 04,8.000,1.360,5.800
2,Arminia Bielefeld,2.680,2.470,3.850
3,VfB Stuttgart,2.470,2.680,3.850
4,Augsburg,17.250,1.130,10.230
5,Bayern Munich,1.130,17.250,10.230
6,Eintracht Frankfurt,2.100,3.100,4.250
7,SC Freiburg,3.100,2.100,4.250
8,Hertha Berlin,3.600,1.945,4.095
9,TSG Hoffenheim,1.945,3.600,4.095


In [40]:
df_teams = pd.read_sql_table('teams', db)

ValueError: Table teams not found

**Old Linear Regression Code**

In [ ]:
# df_player_type = df_player_type.sort_values(by=['matchday'])


# x = np.array(df_player_type['matchday']).reshape((-1, 1))
# y = np.array(df_player_type['count'])
# model = LinearRegression().fit(x, y)
# x_new = np.array(upcoming_matchday).reshape((-1, 1))
# y_pred = model.predict(x_new)
# pred_int = y_pred[0].round(0).astype(int)